In [1]:
import numpy as np
import pandas as pd
import sklearn.ensemble
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor


In [2]:

df_raw = pd.read_csv('train.csv')
df_raw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# #list of categorical and ordinal columns
# categorical_ordinal_col=['Alley',
#  'BldgType',
#  'BsmtCond',
#  'BsmtExposure',
#  'BsmtFinType1',
#  'BsmtFinType2',
#  'BsmtQual',
#  'CentralAir',
#  'Condition1',
#  'Condition2',
#  'Electrical',
#  'ExterCond',
#  'ExterQual',
#  'Exterior1st',
#  'Exterior2nd',
#  'Fence',
#  'FireplaceQu',
#  'Foundation',
#  'Functional',
#  'GarageCond',
#  'GarageFinish',
#  'GarageQual',
#  'GarageType',
#  'Heating',
#  'HeatingQC',
#  'HouseStyle',
#  'KitchenQual',
#  'LandContour',
#  'LandSlope',
#  'LotConfig',
#  'LotShape',
#  'MSSubClass',
#  'MSZoning',
#  'MasVnrType',
#  'MiscFeature',
#  'Neighborhood',
#  'OverallCond',
#  'OverallQual',
#  'PavedDrive',
#  'PoolQC',
#  'RoofMatl',
#  'RoofStyle',
#  'SaleCondition',
#  'SaleType',
#  'Street',
#  'Utilities']



# #List of numerical columns
# numerical_col=['LotFrontage',
#  'LotArea',
#  'YearBuilt',
#  'YearRemodAdd',
#  'MasVnrArea',
#  'BsmtFinSF1',
#  'BsmtFinSF2',
#  'BsmtUnfSF',
#  '1stFlrSF',
#  '2ndFlrSF',
#  'LowQualFinSF',
#  'BedroomAbvGr',
#  'KitchenAbvGr',
#  'TotRmsAbvGrd',
#  'Fireplaces',
#  'GarageYrBlt',
#  'GarageCars',
#  'WoodDeckSF',
#  'OpenPorchSF',
#  'EnclosedPorch',
#  '3SsnPorch',
#  'ScreenPorch',
#  'PoolArea',
#  'MiscVal',
#  'MoSold',
#  'YrSold',
#  'SalePrice',
#  'BsmtBath',
#  'Bath']

In [4]:
import numpy as np
import pandas as pd


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from math import sqrt
from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder


import matplotlib.pyplot as plt
import matplotlib as matplotlib
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')

def data_process(df_raw,remove_outlier = False,remove_hard_to_fit = False,linear_model = False, get_dummies=False, label_encode=False ):

	# Make a copy so the original dataframe will not be altered.
    df_processed = df_raw.copy()
    
    
	# Remove outliers.
    outlier_list = [524, 1299, 463, 31, 534, 1433, 739, 1159, 108, 1231, 971, 1424]
    df_processed = df_processed.drop(outlier_list)

    
    ## Missing values
    
    # 259 LotFrontage  - replace missing value with 0 
    df_processed.LotFrontage = df_processed.LotFrontage.fillna(0)

    # 1369 Alley - replace with None
    df_processed.Alley = df_processed.Alley.fillna('None')

    # 8 MasVnrType and MasVnrArea - replace MasVnrType with None and MasVnrArea with 0
    df_processed.MasVnrType = df_processed.MasVnrType.fillna('None')
    df_processed.MasVnrArea = df_processed.MasVnrArea.fillna(0)

    # 37 basement: BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2- replace with Nb
    df_processed.BsmtQual = df_processed.BsmtQual.fillna('Nb')
    df_processed.BsmtCond = df_processed.BsmtCond.fillna('Nb')
    df_processed.BsmtExposure = df_processed.BsmtExposure.fillna('Nb')
    df_processed.BsmtFinType1 = df_processed.BsmtFinType1.fillna('Nb')
    df_processed.BsmtFinType2 = df_processed.BsmtFinType2.fillna('Nb')
    df_processed.TotalBsmtSF = df_processed.TotalBsmtSF.fillna(0)
    

    # 690 FireplaceQu - replace with Nf
    df_processed.FireplaceQu = df_processed.FireplaceQu.fillna('Nf')

    # 81 Garage: GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond - replace with Ng and year with 0 
    df_processed.GarageType = df_processed.GarageType.fillna('Ng')
    df_processed.GarageFinish = df_processed.GarageFinish.fillna('Ng')
    df_processed.GarageQual = df_processed.GarageQual.fillna('Ng')
    df_processed.GarageCond = df_processed.GarageCond.fillna('Ng')
    df_processed.GarageYrBlt = df_processed.GarageYrBlt.fillna(0)

    # 1453 PoolQC - replace with Np
    df_processed.PoolQC = df_processed.PoolQC.fillna('Np')

    # 1179 Fence - replace with Nf
    df_processed.Fence = df_processed.Fence.fillna('Nf')

    # 1406 MiscFeature - replace with None    
    df_processed.MiscFeature = df_processed.MiscFeature.fillna('None')

    # 1 Electrical
    df_processed = df_processed[pd.notnull(df_processed.Electrical)]

    ## Combine columns and drop multicollinear columns 
    
    # combine bathroom quanlitity 
    df_processed['BsmtBath'] = df_processed.BsmtFullBath + df_processed.BsmtHalfBath * 0.5
    df_processed['Bath'] = df_processed.FullBath + df_processed.HalfBath * 0.5
    df_processed = df_processed.drop(['BsmtFullBath', 'BsmtHalfBath','FullBath','HalfBath'], axis=1)

    # drop TotalBsmtSF - multicollinearaty
    #df_processed = df_processed.drop(['TotalBsmtSF'], axis=1)

    # drop GrLivArea - multicollinearaty
    #df_processed = df_processed.drop(['GrLivArea'], axis=1)

    # drop GarageArea - higher correlation than GarageACars, results are better as well
    df_processed = df_processed.drop(['GarageArea'], axis=1) 
    
    
	# Feature Transformation - take the logarithm of the features.
    #Linear_Num_Cols = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'LotArea', 'GarageArea', 'TotRmsAbvGrd', 'TotalSF', 'BsmtFinSF1']
    df_processed.SalePrice = np.log(df_processed.SalePrice)
    df_processed.GrLivArea = np.log(df_processed.GrLivArea)
    df_processed.TotalBsmtSF = np.log(df_processed.TotalBsmtSF+1)
#     df_processed.LotArea = np.log(df_processed.LotArea) -- performance decreases
#     df_processed.GarageArea = np.log(df_processed.GarageArea)



	# Categorical Features Processsing

	# MSSubClass processing - MSSubClass 20-90 contains only duplicate information with HouseStyle and YearBuilt.
    df_processed['MSSubClass'] = df_processed['MSSubClass'].replace(['20','30','40','45','50','60','70','75','80','85'], '0')

    # Convert numerical to categorical. 
    df_processed[['MSSubClass','OverallQual','OverallCond']] = df_processed[['MSSubClass','OverallQual','OverallCond']].astype(str)

    #Get Dummies 
    
    if get_dummies:
        df_processed = pd.get_dummies(df_processed, columns=df_processed.select_dtypes(include=['object']).columns, drop_first=True)
    
    
    #get label encoder. categorical data change to numerical values
    if label_encode:
        le = LabelEncoder()
        categorical_ordinal_col=df_processed.select_dtypes(include=['object']).columns.to_list()
        df_processed[categorical_ordinal_col]=df_processed[categorical_ordinal_col].apply(lambda col: le.fit_transform(col))

   #---Multiply features: 
#     df_processed["add_OverallGrade"] = df_processed["OverallQual"] * df_processed["OverallCond"]
#     df_processed["add_GarageGrade"] = df_processed["GarageQual"] * df_processed["GarageCond"]
#     df_processed["add_ExterGrade"] = df_processed["ExterQual"] * df_processed["ExterCond"]
#     df_processed["add_KitchenScore"] = df_processed["KitchenAbvGr"] * df_processed["KitchenQual"]
#     df_processed["add_FireplaceScore"] = df_processed["Fireplaces"] * df_processed["FireplaceQu"]
#     df_processed["add_GarageScore"] = df_processed["GarageArea"] * df_processed["GarageQual"]
#     df_processed["add_PoolScore"] = df_processed["PoolArea"] * df_processed["PoolQC"]
#     df_processed['add_GrLivArea*OvQual'] = df_processed['GrLivArea'] * df_processed['OverallQual']
#     df_processed['add_QualOverall*Exter*Kitch*Bsmt*Garage'] = df_processed['OverallQual'] * df_processed['ExterQual'] * df_processed['KitchenQual'] * df_processed['BsmtQual'] * df_processed['GarageQual']



    return df_processed



In [5]:
#data prep
data_processed_label_encode=data_process(df_raw,label_encode=True)
data_processed_label_encode.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,BsmtBath,Bath
0,1,9,3,65.0,8450,1,1,3,3,0,...,4,1,0,2,2008,8,4,12.247694,1.0,2.5
1,2,4,3,80.0,9600,1,1,3,3,0,...,4,1,0,5,2007,8,4,12.109011,0.5,2.0
2,3,9,3,68.0,11250,1,1,0,3,0,...,4,1,0,9,2008,8,4,12.317167,1.0,2.5
3,4,10,3,60.0,9550,1,1,0,3,0,...,4,1,0,2,2006,8,0,11.849398,1.0,1.0
4,5,9,3,84.0,14260,1,1,0,3,0,...,4,1,0,12,2008,8,4,12.429216,1.0,2.5


In [6]:
# A boosting tree is an ensemble of trees, thus it inherits all the hyperparameters of decision trees

#     max_depth: max tree height, default value 3
#     max_leaf_nodes
#     min_samples_leaf
#     min_samples_split
#     min_impurity_decrease
#     min_impurity_split
#     criterion: the criterion for tree splitting, defaulted to friedman_mse. friedman_mse is the same as mse (mean square error) when the loss function is chosen to be ls (least square)

# On the other hand, there are ensemble-specific hyperparameters

#     n_estimators: the number of trees, default value 100
#     learning_rate: the shrinkage/discount factor, default value 0.1
#     subsamples: the sample count in term of percentage used for a single tree training
#     max_features: the integer or percentage count of the number of random feature selected for one tree node



In [7]:
# def GradientBoosting(data_processed):

    
#     def eval_metrics(actual, pred):
#         rmse = np.sqrt(mean_squared_error(actual, pred))
#         mae = mean_absolute_error(actual, pred)
#         r2 = r2_score(actual, pred)
#         return rmse, mae, r2

#     gbm = GradientBoostingRegressor(learning_rate=0.1, n_estimators, max_depth, min_samples_split, min_samples_leaf  )
#     #n_estimators: for each residual, i do a tree. this is how many tres that i have. 
    
#    # The number of boosting stages to perform. Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance.

    
#     # Split the data into training and test sets. (0.75, 0.25) split.
#     train, test = train_test_split(data_processed)

#     # The predicted column is "SalePrice" .
#     train_x = train.drop(["SalePrice"], axis=1)
#     test_x = test.drop(["SalePrice"], axis=1)
#     train_y = train[["SalePrice"]]
#     test_y = test[["SalePrice"]]
    
    

#     # Fit the model 
#     gbm.fit(train_x,train_y.values.ravel())
    
#     print('-'*50)
#     print("The train set R^2 is %.3f" %(gbm.score(train_x, train_y)))
#     print("The test set R^2 is %.3f" %(gbm.score(test_x,test_y)))
    
#     y_pred=gbm.predict(test_x)
    
#     #eval_metrics(actual, predicted)
#     (rmse, mae, r2) = eval_metrics(test_y, y_pred)
    
#     # Print out metrics
#     print("  RMSE: %s" % rmse)
#     print("  MAE: %s" % mae)
#     print("  R2: %s" % r2)
    
#     print('feature importance:', gbm.feature_importances_)
    
#     return 


In [8]:

# GradientBoosting(data_processed_with_dummy)

In [10]:
#random search with gradient boosting regressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
num_estimators = [500,1000]
learn_rates = [0.02, 0.05]
max_depths = [1, 2]
min_samples_leaf = [5,10]
min_samples_split = [5,10]

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data_processed_label_encode)

# The predicted column is "SalePrice" .
train_x = train.drop(["SalePrice"], axis=1)
test_x = test.drop(["SalePrice"], axis=1)
train_y = train[["SalePrice"]]
test_y = test[["SalePrice"]]

param_grid = {'n_estimators': num_estimators,
              'learning_rate': learn_rates,
              'max_depth': max_depths,
              'min_samples_leaf': min_samples_leaf,
              'min_samples_split': min_samples_split}

random_search =RandomizedSearchCV(GradientBoostingRegressor(loss='huber'), param_grid, random_state=1, n_iter=100, cv=5, verbose=0, n_jobs=-1)

random_search.fit(train_x, train_y)
random_search.best_params_


KeyboardInterrupt: 

In [ ]:
#grfrom sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

num_estimators = [1000]
learn_rates = [0.02, 0.05]
max_depths = [1,2]
min_samples_leaf = [5,10]
min_samples_split = [5,10]

param_grid = {'n_estimators': num_estimators,
              'learning_rate': learn_rates,
              'max_depth': max_depths,
              'min_samples_leaf': min_samples_leaf,
              'min_samples_split': min_samples_split}

grid_search = GridSearchCV(GradientBoostingRegressor(loss='huber'),
                           param_grid, cv=3, return_train_score=True)

grid_search.fit(train_x, train_y.values.ravel())